In [1]:
import tensorflow as tf
import pandas as pd
import json
import configparser
from pathlib import Path
import os
    
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import MaxPooling1D

configs = configparser.ConfigParser()
configs.read("configuracoes.conf",encoding='utf-8')

data_folder = Path(configs["DEFAULT"]["folder_path"])
path = data_folder / configs["DEFAULT"]["file"]
objeto = configs["DEFAULT"]["objeto"]
regiao = configs["DEFAULT"]["regiao"]
look_back = 12 #Quantidade de meses anteriores usado para prever
forecast_horizon = int(configs["DEFAULT"]["meses"]) #Quantidade de meses previstos

# Prever Cesta

In [ ]:
#Carrega os dados
cesta = pd.read_excel(path)
cesta.drop(['ano', 'salario', 'razao'], axis=1, inplace=True)
cesta = cesta / 1000
cesta = cesta.iloc[::, ::]

#Cria sequencias de tempo de tamanho (meses anteriores + meses a serem previstos)
for n_step in range(1, look_back + forecast_horizon):
    cesta[f'preco t(h + {n_step})'] = cesta['preco'].shift(-n_step).values
cesta.dropna(inplace=True)
cesta.reset_index(drop=True, inplace=True)

#Divide os dados em Treino e Teste
X_train = cesta.iloc[::, :look_back:].values
y_train = cesta.iloc[::, look_back::].values
X_val = cesta.iloc[-1::, -12::].values

#remodela os dados para formato (batches, sequencia de tempo, features)
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_val = X_val.reshape((X_val.shape[0], X_val.shape[1], 1))

#Cria o modelo
model=Sequential()
model.add(Conv1D(filters=24, kernel_size=3, input_shape=(look_back,1), activation='tanh'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(15, activation='tanh'))
model.add(Dense(forecast_horizon))
model.compile(loss='mean_squared_error', optimizer=tf.keras.optimizers.Adam(learning_rate=0.0003))

#Treina e prever (forecast_horizon) quantidade de tempo
history = model.fit(X_train, y_train, epochs= 150, batch_size=1, shuffle=False)
forecast = model.predict(X_val, batch_size=1)

resultados = [valor for valor in forecast[0]]
objeto = objeto.replace(" ",'_')
output = {objeto.lower():str(resultados)}
filename = f"previsao_CNN_{objeto.lower()}_{regiao.lower()}.json"
pasta_atual = os.getcwd()
full_url = Path(pasta_atual) / 'resultados' / 'previsoes_cesta' / filename
os.makedirs(os.path.dirname(full_url), exist_ok=True)
with open(full_url,"w") as file:
    json.dump(output,file,ensure_ascii=False)
print("Arquivo com resultados gerados")


# Prever Produtos

In [2]:
path_inicial = r'C:\\Users\\mathe\\Documents\\NotebooksMateriais\\previsao_cestas\\datasets_produtos\\'

regioes = ['ilheus','itabuna']
produtos = ['acucar', 'arroz', 'banana', 'cafe', 'carne', 'farinha', 'feijao', 'leite', 'manteiga', 'oleo', 'pao', 'tomate']

for regiao in regioes:
    for objeto in produtos:
        #Carrega os dados
        path = f'{path_inicial}{regiao}/{objeto}_{regiao}.xlsx'
        cesta = pd.read_excel(path)
        cesta.drop(['ano'], axis=1, inplace=True)
        cesta = cesta / 1000
        cesta = cesta.iloc[::]

        #Cria sequencias de tempo de tamanho (meses anteriores + meses a serem previstos)
        for n_step in range(1, look_back + forecast_horizon):
            cesta[f'preco t(h + {n_step})'] = cesta['preco'].shift(-n_step).values
        cesta.dropna(inplace=True)
        cesta.reset_index(drop=True, inplace=True)

        #Divide os dados em Treino e Teste
        X_train = cesta.iloc[::, :look_back:].values
        y_train = cesta.iloc[::, look_back::].values
        X_val = cesta.iloc[-1::, -12::].values

        #remodela os dados para formato (batches, sequencia de tempo, features)
        X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
        X_val = X_val.reshape((X_val.shape[0], X_val.shape[1], 1))

        #Cria o modelo
        model=Sequential()
        model.add(Conv1D(filters=24, kernel_size=3, input_shape=(look_back,1), activation='tanh'))
        model.add(MaxPooling1D(pool_size=2))
        model.add(Flatten())
        model.add(Dense(15, activation='tanh'))
        model.add(Dense(forecast_horizon))
        model.compile(loss='mean_squared_error', optimizer=tf.keras.optimizers.Adam(learning_rate=0.0003))

        #Treina e prever (forecast_horizon) quantidade de tempo
        history = model.fit(X_train, y_train, epochs= 150, batch_size=1, shuffle=False)
        forecast = model.predict(X_val, batch_size=1)

        resultados = [valor for valor in forecast[0]]
        objeto = objeto.replace(" ", '_')
        output = {objeto.lower():str(resultados)}
        filename = f"previsao_CNN_{objeto.lower()}_{regiao.lower()}.json"
        pasta_atual = os.getcwd()
        full_url = Path(pasta_atual) / 'resultados' / 'previsoes_produtos' / f'{regiao}' /filename
        os.makedirs(os.path.dirname(full_url), exist_ok=True)
        with open(full_url,"w") as file:
            json.dump(output,file,ensure_ascii=False)
        print("Arquivo com resultados gerados")

Epoch 1/150
197/197 [==============================] - 0s 458us/step - loss: 1.7430e-06
Epoch 2/150
197/197 [==============================] - 0s 481us/step - loss: 4.5955e-06
Epoch 3/150
197/197 [==============================] - 0s 486us/step - loss: 3.2503e-06
Epoch 4/150
197/197 [==============================] - 0s 501us/step - loss: 3.2404e-06
Epoch 5/150
197/197 [==============================] - 0s 582us/step - loss: 3.3118e-06
Epoch 6/150
197/197 [==============================] - 0s 435us/step - loss: 3.6877e-06
Epoch 7/150
197/197 [==============================] - 0s 466us/step - loss: 3.4362e-06
Epoch 8/150
197/197 [==============================] - 0s 501us/step - loss: 4.2568e-06
Epoch 9/150
197/197 [==============================] - 0s 435us/step - loss: 3.6831e-06
Epoch 10/150
197/197 [==============================] - 0s 521us/step - loss: 4.6208e-06
Epoch 11/150
197/197 [==============================] - 0s 451us/step - loss: 3.2661e-06
Epoch 12/150
197/197 [========

197/197 [==============================] - 0s 491us/step - loss: 5.2097e-06
Epoch 93/150
197/197 [==============================] - 0s 491us/step - loss: 5.0810e-06
Epoch 94/150
197/197 [==============================] - 0s 608us/step - loss: 4.9921e-06
Epoch 95/150
197/197 [==============================] - 0s 532us/step - loss: 5.0440e-06
Epoch 96/150
197/197 [==============================] - 0s 501us/step - loss: 5.0532e-06
Epoch 97/150
197/197 [==============================] - 0s 466us/step - loss: 4.9875e-06
Epoch 98/150
197/197 [==============================] - 0s 484us/step - loss: 4.9701e-06
Epoch 99/150
197/197 [==============================] - 0s 501us/step - loss: 4.9565e-06
Epoch 100/150
197/197 [==============================] - 0s 542us/step - loss: 4.8869e-06
Epoch 101/150
197/197 [==============================] - 0s 529us/step - loss: 4.8568e-06
Epoch 102/150
197/197 [==============================] - 0s 615us/step - loss: 4.8886e-06
Epoch 103/150
197/197 [========

197/197 [==============================] - 0s 552us/step - loss: 4.8996e-06
Epoch 34/150
197/197 [==============================] - 0s 506us/step - loss: 5.7764e-06
Epoch 35/150
197/197 [==============================] - 0s 496us/step - loss: 4.9388e-06
Epoch 36/150
197/197 [==============================] - 0s 547us/step - loss: 5.9436e-06
Epoch 37/150
197/197 [==============================] - 0s 577us/step - loss: 4.6722e-06
Epoch 38/150
197/197 [==============================] - 0s 503us/step - loss: 6.2699e-06
Epoch 39/150
197/197 [==============================] - 0s 521us/step - loss: 5.1046e-06
Epoch 40/150
197/197 [==============================] - 0s 527us/step - loss: 5.4928e-06
Epoch 41/150
197/197 [==============================] - 0s 499us/step - loss: 4.9341e-06
Epoch 42/150
197/197 [==============================] - 0s 500us/step - loss: 5.9168e-06
Epoch 43/150
197/197 [==============================] - 0s 501us/step - loss: 5.1878e-06
Epoch 44/150
197/197 [============

197/197 [==============================] - 0s 506us/step - loss: 5.1030e-06
Epoch 125/150
197/197 [==============================] - 0s 577us/step - loss: 4.4217e-06
Epoch 126/150
197/197 [==============================] - 0s 509us/step - loss: 4.8697e-06
Epoch 127/150
197/197 [==============================] - 0s 497us/step - loss: 4.4653e-06
Epoch 128/150
197/197 [==============================] - 0s 512us/step - loss: 4.5787e-06
Epoch 129/150
197/197 [==============================] - 0s 567us/step - loss: 4.7222e-06
Epoch 130/150
197/197 [==============================] - 0s 504us/step - loss: 4.9697e-06
Epoch 131/150
197/197 [==============================] - 0s 509us/step - loss: 4.6619e-06
Epoch 132/150
197/197 [==============================] - 0s 501us/step - loss: 4.8103e-06
Epoch 133/150
197/197 [==============================] - 0s 501us/step - loss: 4.4316e-06
Epoch 134/150
197/197 [==============================] - 0s 506us/step - loss: 4.4386e-06
Epoch 135/150
197/197 [=

197/197 [==============================] - 0s 457us/step - loss: 4.0483e-05
Epoch 66/150
197/197 [==============================] - 0s 406us/step - loss: 4.0373e-05
Epoch 67/150
197/197 [==============================] - 0s 558us/step - loss: 4.0228e-05
Epoch 68/150
197/197 [==============================] - 0s 410us/step - loss: 4.0122e-05
Epoch 69/150
197/197 [==============================] - 0s 429us/step - loss: 3.9979e-05
Epoch 70/150
197/197 [==============================] - 0s 505us/step - loss: 3.9870e-05
Epoch 71/150
197/197 [==============================] - 0s 406us/step - loss: 3.9740e-05
Epoch 72/150
197/197 [==============================] - 0s 407us/step - loss: 3.9623e-05
Epoch 73/150
197/197 [==============================] - 0s 506us/step - loss: 3.9506e-05
Epoch 74/150
197/197 [==============================] - 0s 408us/step - loss: 3.9384e-05
Epoch 75/150
197/197 [==============================] - 0s 406us/step - loss: 3.9276e-05
Epoch 76/150
197/197 [============

...

197/197 [==============================] - 0s 577us/step - loss: 2.4768e-06
Epoch 82/150
197/197 [==============================] - 0s 734us/step - loss: 2.4518e-06
Epoch 83/150
197/197 [==============================] - 0s 749us/step - loss: 2.3987e-06
Epoch 84/150
197/197 [==============================] - 0s 572us/step - loss: 2.4809e-06 0s - loss: 2.6096e-0
Epoch 85/150
197/197 [==============================] - 0s 623us/step - loss: 2.3604e-06
Epoch 86/150
197/197 [==============================] - ETA: 0s - loss: 2.7261e-0 - 0s 638us/step - loss: 2.5102e-06
Epoch 87/150
197/197 [==============================] - 0s 628us/step - loss: 2.6268e-06
Epoch 88/150
197/197 [==============================] - 0s 572us/step - loss: 2.4427e-06
Epoch 89/150
197/197 [==============================] - 0s 577us/step - loss: 2.7069e-06
Epoch 90/150
197/197 [==============================] - 0s 666us/step - loss: 2.2786e-06
Epoch 91/150
197/197 [==============================] - 0s 640us/step - lo

197/197 [==============================] - 0s 597us/step - loss: 1.4177e-05
Epoch 14/150
197/197 [==============================] - 0s 582us/step - loss: 1.3775e-05
Epoch 15/150
197/197 [==============================] - 0s 752us/step - loss: 1.3527e-05
Epoch 16/150
197/197 [==============================] - 0s 972us/step - loss: 1.3392e-05
Epoch 17/150
197/197 [==============================] - 0s 795us/step - loss: 1.3325e-05
Epoch 18/150
197/197 [==============================] - 0s 643us/step - loss: 1.3284e-05
Epoch 19/150
197/197 [==============================] - 0s 628us/step - loss: 1.3247e-05
Epoch 20/150
197/197 [==============================] - 0s 602us/step - loss: 1.3203e-05
Epoch 21/150
197/197 [==============================] - 0s 1ms/step - loss: 1.3129e-05
Epoch 22/150
197/197 [==============================] - 0s 547us/step - loss: 1.3046e-05
Epoch 23/150
197/197 [==============================] - 0s 527us/step - loss: 1.2934e-05
Epoch 24/150
197/197 [==============

Epoch 105/150
197/197 [==============================] - 0s 455us/step - loss: 8.9600e-06
Epoch 106/150
197/197 [==============================] - 0s 469us/step - loss: 8.9438e-06
Epoch 107/150
197/197 [==============================] - 0s 509us/step - loss: 8.9219e-06
Epoch 108/150
197/197 [==============================] - 0s 509us/step - loss: 8.9012e-06
Epoch 109/150
197/197 [==============================] - 0s 508us/step - loss: 8.8803e-06
Epoch 110/150
197/197 [==============================] - 0s 494us/step - loss: 8.8591e-06
Epoch 111/150
197/197 [==============================] - 0s 648us/step - loss: 8.8389e-06
Epoch 112/150
197/197 [==============================] - 0s 457us/step - loss: 8.8214e-06
Epoch 113/150
197/197 [==============================] - 0s 508us/step - loss: 8.8083e-06
Epoch 114/150
197/197 [==============================] - 0s 456us/step - loss: 8.7976e-06
Epoch 115/150
197/197 [==============================] - 0s 508us/step - loss: 8.7810e-06
Epoch 116/

197/197 [==============================] - 0s 507us/step - loss: 1.3528e-04
Epoch 38/150
197/197 [==============================] - 0s 456us/step - loss: 1.3487e-04
Epoch 39/150
197/197 [==============================] - 0s 456us/step - loss: 1.3447e-04
Epoch 40/150
197/197 [==============================] - 0s 406us/step - loss: 1.3408e-04
Epoch 41/150
197/197 [==============================] - 0s 407us/step - loss: 1.3370e-04
Epoch 42/150
197/197 [==============================] - 0s 405us/step - loss: 1.3333e-04
Epoch 43/150
197/197 [==============================] - 0s 407us/step - loss: 1.3297e-04
Epoch 44/150
197/197 [==============================] - 0s 405us/step - loss: 1.3262e-04
Epoch 45/150
197/197 [==============================] - 0s 405us/step - loss: 1.3228e-04
Epoch 46/150
197/197 [==============================] - 0s 405us/step - loss: 1.3194e-04
Epoch 47/150
197/197 [==============================] - 0s 407us/step - loss: 1.3161e-04
Epoch 48/150
197/197 [============

197/197 [==============================] - 0s 405us/step - loss: 1.1473e-04
Epoch 129/150
197/197 [==============================] - 0s 459us/step - loss: 1.1460e-04
Epoch 130/150
197/197 [==============================] - 0s 664us/step - loss: 1.1447e-04
Epoch 131/150
197/197 [==============================] - 0s 452us/step - loss: 1.1436e-04
Epoch 132/150
197/197 [==============================] - 0s 457us/step - loss: 1.1423e-04
Epoch 133/150
197/197 [==============================] - 0s 457us/step - loss: 1.1411e-04
Epoch 134/150
197/197 [==============================] - 0s 456us/step - loss: 1.1399e-04
Epoch 135/150
197/197 [==============================] - 0s 407us/step - loss: 1.1387e-04
Epoch 136/150
197/197 [==============================] - 0s 406us/step - loss: 1.1375e-04
Epoch 137/150
197/197 [==============================] - 0s 406us/step - loss: 1.1364e-04
Epoch 138/150
197/197 [==============================] - 0s 406us/step - loss: 1.1352e-04
Epoch 139/150
197/197 [=